## Introduction

This workbook contains data wrangling for Making the News on r/news. Data were imported and optimized for use in machine learning algorithms. For this reason, most data were either left as or converted to integer format. 

### Table of Contents

1. [Imports](#Imports)
2. [Data uploading](#Uploading)
3. [Removing duplicates](#DeDup)
4. [Create engagement measure](#Engagement)
5. [Set engagement range](#EngRange)
6. [Export!](#export)

### 1. Imports <a class="anchor" id="Imports"></a>

In [1]:
import pandas as pd
import numpy as np
import datetime as dt

### 2. Data uploading <a class="anchor" id="Uploading"></a>

In [2]:
# Load data and drop redundant columns

data = pd.read_csv('r_news_data.csv')
data = data.drop(['Unnamed: 0', 'level_0', 'index'], axis=1)
data.head()

,Post ID,Title,Url,Author,Score,Publish Date,Total No. of Comments,Permalink,Flair
0,ko10lt,Brexit Becomes Reality As UK Leaves European U...,https://popularnews.in/brexit-becomes-reality-...,popularnewsindia,1,2020-12-31 16:00:01,0,/r/news/comments/ko10lt/brexit_becomes_reality...,NaN
1,ko10oh,COVID-19 UPDATE: Take-out and Delivery OPEN. N...,https://siliconeer.com/current/raj-palace-rest...,siliconeer,1,2020-12-31 16:00:06,0,/r/news/comments/ko10oh/covid19_update_takeout...,NaN
2,ko10sx,Lamborghini Huracan STO Teased Ahead Of Debut,https://popularnews.in/lamborghini-huracan-sto...,popularnewsindia,1,2020-12-31 16:00:15,0,/r/news/comments/ko10sx/lamborghini_huracan_st...,NaN
3,ko11ms,"""It's just utter chaos"": California becomes th...",https://www.cbsnews.com/news/california-become...,Bonboniru,1,2020-12-31 16:01:21,269,/r/news/comments/ko11ms/its_just_utter_chaos_c...,NaN
4,ko11u9,The Number Theory| A decade of rightward shift...,https://popularnews.in/the-number-theory-a-dec...,popularnewsindia,1,2020-12-31 16:01:38,0,/r/news/comments/ko11u9/the_number_theory_a_de...,NaN


### 3. Removing duplicates<a class="anchor" id="DeDup"></a>

In [3]:
# Split data into duplicate URLs and non-duplicates
# Made the decision to Sum scores and comments
# This way we still treat separte engagements as measures of virality

data_dup = pd.DataFrame(data)
data_dup['duplicate'] = data_dup.duplicated(subset=['Url'], keep=False)
data_dedup = data_dup[data_dup['duplicate'] == True]
data_solo = data_dup[data_dup['duplicate'] == False]

In [4]:
# Sum score, comments for duplicate URLs

data_dedup_urls = data_dedup.groupby(by=['Url']).sum()

In [5]:
# Rejoin Sums with dataset again
# Final result is a deduplicated data set with aggregated Scores and # Comments
# Publish Date was kept as first posted, not most recent
# Most appear to have been ads

data_dedup = data_dedup.join(data_dedup_urls, on=['Url'], how='right',rsuffix='_dup')
data_dedup = data_dedup.drop(['Score', 'Total No. of Comments', 'duplicate', 'duplicate_dup'], axis=1)
data_dedup = data_dedup[['Post ID','Title','Url','Author','Score_dup','Publish Date','Total No. of Comments_dup','Permalink','Flair']]
data_dedup = data_dedup.rename(columns={'Score_dup':"Score","Total No. of Comments_dup":'Total No. of Comments'})
data_dedup['Publish Date'] = pd.to_datetime(data_dedup['Publish Date'])
data_dedup.sort_values(by="Publish Date", inplace=True)
data_dedup.drop_duplicates(subset='Url', keep='first', inplace=True)

In [6]:
# Append deduped cells back to unique cells
# Left in duplicate boolean in case it's interesting

data = pd.DataFrame(data_solo.append(data_dedup, ignore_index=True))
data['duplicate'] = data['duplicate'].fillna(True)

### 4. Create engagement measure <a class="anchor" id="Engagement"></a>

In [7]:
# Create aggregate column for Score and Comments, to see what has effectively zero engagement
data['Engagement'] = data['Score'] + data['Total No. of Comments']

In [8]:
data

,Post ID,Title,Url,Author,Score,Publish Date,Total No. of Comments,Permalink,Flair,duplicate,Engagement
0,ko10lt,Brexit Becomes Reality As UK Leaves European U...,https://popularnews.in/brexit-becomes-reality-...,popularnewsindia,1,2020-12-31 16:00:01,0,/r/news/comments/ko10lt/brexit_becomes_reality...,NaN,False,1
1,ko10sx,Lamborghini Huracan STO Teased Ahead Of Debut,https://popularnews.in/lamborghini-huracan-sto...,popularnewsindia,1,2020-12-31 16:00:15,0,/r/news/comments/ko10sx/lamborghini_huracan_st...,NaN,False,1
2,ko11ms,"""It's just utter chaos"": California becomes th...",https://www.cbsnews.com/news/california-become...,Bonboniru,1,2020-12-31 16:01:21,269,/r/news/comments/ko11ms/its_just_utter_chaos_c...,NaN,False,270
3,ko11u9,The Number Theory| A decade of rightward shift...,https://popularnews.in/the-number-theory-a-dec...,popularnewsindia,1,2020-12-31 16:01:38,0,/r/news/comments/ko11u9/the_number_theory_a_de...,NaN,False,1
4,ko12dy,President Buhari To Address The Nation on The ...,"https://www.gistmania.com/talk/topic,483033.0....",Gistmania,1,2020-12-31 16:02:33,0,/r/news/comments/ko12dy/president_buhari_to_ad...,NaN,False,1
...,...,...,...,...,...,...,...,...,...,...,...
225578,n2mad1,UV Sanitizer Light - Portable Ultraviolet Ligh...,https://runshinemall.com/collections/uv-saniti...,Status_Pound_9962,2,2021-05-01 09:54:04,0,/r/news/comments/n2mad1/uv_sanitizer_light_por...,NaN,True,2
225579,n3a82s,Cities and states across the US are promising ...,https://www.yahoo.com/news/cities-states-acros...,shallah,2,2021-05-02 09:36:48,0,/r/news/comments/n3a82s/cities_and_states_acro...,NaN,True,2
225580,n48ptn,"Pete Lammons, Who Helped the Jets Win ’69 Trem...",https://newsnationglobal.com/2021/05/03/pete-l...,newsnationglobal,2,2021-05-03 15:04:43,0,/r/news/comments/n48ptn/pete_lammons_who_helpe...,NaN,True,2
225581,n5lg39,National Guard soldier charged for storming th...,https://www.cnn.com/2021/05/05/politics/wiscon...,7MCMXC,2,2021-05-05 10:26:45,3348,/r/news/comments/n5lg39/national_guard_soldier...,NaN,True,3350


In [9]:
# Sort range, with highest Engagement on top

data = pd.DataFrame(data.sort_values(by='Engagement', ascending=False)).reset_index(drop=True)

### 5. Set engagement range <a class="anchor" id="EngRange"></a>

In [10]:
# Set Engagement cut point to 10 

data_top_engagement = data[data['Engagement'] > 9]

In [13]:
data_top_engagement

,Post ID,Title,Url,Author,Score,Publish Date,Total No. of Comments,Permalink,Flair,duplicate,Engagement
0,lr3xap,Man dies after police kneel on his neck for ne...,https://amp.cnn.com/cnn/2021/02/23/us/angelo-q...,monaleeparis,100226,2021-02-23 20:43:30,17754,/r/news/comments/lr3xap/man_dies_after_police_...,NaN,False,117980
1,lsmh36,Texan files $1 billion class-action lawsuit af...,https://abcnews.go.com/US/texan-files-billion-...,ACABBLM2020,97551,2021-02-25 17:30:11,14574,/r/news/comments/lsmh36/texan_files_1_billion_...,NaN,False,112125
2,ls7xmj,Trump tax returns are now in the hands of the ...,https://www.cnbc.com/2021/02/25/trump-tax-retu...,[deleted],98318,2021-02-25 06:38:18,12719,/r/news/comments/ls7xmj/trump_tax_returns_are_...,NaN,True,111037
3,laun1r,Minneapolis police officers must keep body cam...,https://www.cnn.com/2021/02/02/us/minneapolis-...,dlkapt3,103302,2021-02-02 04:42:35,7312,/r/news/comments/laun1r/minneapolis_police_off...,NaN,False,110614
4,lsbgop,Costco lifts minimum wage above Amazon or Targ...,https://www.reuters.com/article/us-costco-whol...,candordirect,99299,2021-02-25 09:11:08,8124,/r/news/comments/lsbgop/costco_lifts_minimum_w...,NaN,False,107423
...,...,...,...,...,...,...,...,...,...,...,...
11235,mxf36e,Bones of Black children killed in police bombi...,https://www.theguardian.com/us-news/2021/apr/2...,mod_89,1,2021-04-24 00:35:28,9,/r/news/comments/mxf36e/bones_of_black_childre...,NaN,False,10
11236,mjb8qw,A juvenile has been arrested following a video...,https://www.cnn.com/2021/04/03/us/asian-couple...,Piranha_ChuckNorris,5,2021-04-03 08:57:30,5,/r/news/comments/mjb8qw/a_juvenile_has_been_ar...,NaN,False,10
11237,n3oquf,(Australia) 80yo fisher finds himself trapped ...,https://www.abc.net.au/news/2021-05-03/fisherm...,LuckyBdx4,1,2021-05-02 22:29:06,9,/r/news/comments/n3oquf/australia_80yo_fisher_...,NaN,False,10
11238,n545c8,Minorities underrepresented in vaccinated popu...,https://www.kxly.com/minorities-underrepresent...,MasterRazz,1,2021-05-04 18:40:19,9,/r/news/comments/n545c8/minorities_underrepres...,NaN,False,10


### 6. Export! <a class="anchor" id="export"></a>

In [11]:
# Save datasets to csv

data_top_engagement.to_csv('r_news_top_jan_to_may_2021.csv')
data.to_csv('r_news_all_jan_to_may_2021.csv')